## Comando para baixar o PDF

In [3]:
!TOKENIZERS_PARALLELISM=true wget -O biblioteca.pdf "https://anexosportal.datalegis.net/arquivos/1878261.pdf"

--2025-04-12 17:27:27--  https://anexosportal.datalegis.net/arquivos/1878261.pdf
Resolving anexosportal.datalegis.net (anexosportal.datalegis.net)... 104.21.96.1, 104.21.112.1, 104.21.80.1, ...
Connecting to anexosportal.datalegis.net (anexosportal.datalegis.net)|104.21.96.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455898 (445K) [application/pdf]
Saving to: ‘biblioteca.pdf’

biblioteca.pdf      100%[===================>] 445.21K  --.-KB/s    in 0.05s   

2025-04-12 17:27:27 (9.27 MB/s) - ‘biblioteca.pdf’ saved [455898/455898]



In [ ]:
!pip install langchain faiss-cpu sentence-transformers
!pip install langchain-community
!pip install langchain_huggingface
!pip install pypdf
!pip install chromadb
!pip install langchain_openai



## Código para popular um Banco de Dados Vetorial

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

dados = PyPDFLoader("biblioteca.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine)

## Código para montar um RAG

In [26]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from dotenv import load_dotenv
import os

from google.colab import userdata
#userdata.get('OPEN_API_KEY')
#load_dotenv()

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
n_documentos = 3

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos)
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Código para perguntar ao RAG

In [ ]:
prompt = "Qual a norma que trata de seringas hipodérmicas?"
rag.invoke(prompt)

'A norma que trata de seringas hipodérmicas é a Portaria INMETRO 458/2021, que aprova os Requisitos de Avaliação da Conformidade para Seringas Hipodérmicas Estéreis de Uso Único.'